In [37]:
import pandas as pd
import numpy as np

In [38]:
dataset=pd.read_csv(r"D:\End to end project\Moody_Chatbot\mOODY_cHATBOT\emotions.csv\emotions.csv")
dataset.head()

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [39]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    416809 non-null  object
 1   label   416809 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.4+ MB


In [40]:
print(dataset.isnull().sum())
dataset.dropna(inplace=True)
print(dataset.isnull().sum())
for i in dataset.duplicated():
    if i=="True":
        print(i)


text     0
label    0
dtype: int64
text     0
label    0
dtype: int64


In [41]:
# Here we lowercase our text column
dataset['text']=dataset['text'].str.lower()

In [42]:
# Here we remove the punctuation from the text
import string, time
exclude=string.punctuation
def remove_puch(text):
    text.translate(str.maketrans('','',exclude))
dataset['text'].apply(remove_puch)

0         None
1         None
2         None
3         None
4         None
          ... 
416804    None
416805    None
416806    None
416807    None
416808    None
Name: text, Length: 416809, dtype: object

In [43]:
dataset

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
416804,i feel like telling these horny devils to find...,2
416805,i began to realize that when i was feeling agi...,3
416806,i feel very curious be why previous early dawn...,5
416807,i feel that becuase of the tyranical nature of...,3


In [44]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
words=stopwords.words('english')

def remove_stopword(text):
    new_text=[]
    for word in text.split():
        if word in words:
            new_text.append(" ")
        else:
            new_text.append(word)
    x=new_text[ : ]
    new_text.clear()
    return " ".join(x)

type(dataset['text'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


pandas.core.series.Series

In [45]:
# Here we perfrom stemming to text to convert the text to their root words
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stemmer(text):
  stem_list=[]
  for word in text.split():
    stem_list.append(ps.stem(word))
  return " ".join(stem_list)
dataset['text']=dataset['text'].apply(stemmer)


In [46]:
dataset

,text,label
0,i just feel realli helpless and heavi heart,4
1,ive enjoy be abl to slouch about relax and unw...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
416804,i feel like tell these horni devil to find a s...,2
416805,i began to realiz that when i wa feel agit or ...,3
416806,i feel veri curiou be whi previou earli dawn i...,5
416807,i feel that becuas of the tyran natur of the g...,3


**TFidf**

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

x_train, x_test, y_train, y_test= train_test_split(
    dataset['text'], dataset['label'], test_size=0.2, random_state=42
)

vectorizer = TfidfVectorizer(stop_words='english')
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr_tfidf=LogisticRegression(max_iter=2000)
lr_tfidf.fit(x_train_tfidf,y_train)

y_pred=lr_tfidf.predict(x_test_tfidf)
accuracy_lr_tfidf=accuracy_score(y_test,y_pred)
print(accuracy_lr_tfidf)

0.8735874859048487


**Word2Vec**

In [15]:
# Here we tokenize our text
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
dataset['text']=dataset['text'].apply(word_tokenize)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [16]:
dataset

,text,label
0,"[i, just, feel, realli, helpless, and, heavi, ...",4
1,"[ive, enjoy, be, abl, to, slouch, about, relax...",0
2,"[i, gave, up, my, internship, with, the, dmrg,...",4
3,"[i, dont, know, i, feel, so, lost]",0
4,"[i, am, a, kindergarten, teacher, and, i, am, ...",4
...,...,...
416804,"[i, feel, like, tell, these, horni, devil, to,...",2
416805,"[i, began, to, realiz, that, when, i, wa, feel...",3
416806,"[i, feel, veri, curiou, be, whi, previou, earl...",5
416807,"[i, feel, that, becuas, of, the, tyran, natur,...",3


In [17]:
# Here we convert our text into Dense Vector by Word2Vec technique
from gensim.models import Word2Vec
model = Word2Vec(sentences=dataset['text'], vector_size=100, window=5, min_count=1, workers=4)


In [18]:
# Here we find the average of vectors
def sentence_vector(words, model):
    # Only keep words that exist in model vocab
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

dataset['vector'] = dataset['text'].apply(lambda x: sentence_vector(x, model))


In [19]:
dataset

,text,label,vector
0,"[i, just, feel, realli, helpless, and, heavi, ...",4,"[0.6494968, 0.06028536, -0.5887667, -0.0268500..."
1,"[ive, enjoy, be, abl, to, slouch, about, relax...",0,"[0.08854502, -0.13225561, -0.16946371, -0.3908..."
2,"[i, gave, up, my, internship, with, the, dmrg,...",4,"[0.2870616, 0.10906085, -0.050448354, -0.16801..."
3,"[i, dont, know, i, feel, so, lost]",0,"[0.2361146, -0.31551522, -0.70028865, 0.349299..."
4,"[i, am, a, kindergarten, teacher, and, i, am, ...",4,"[0.11820791, -0.099037394, -0.019013226, 0.188..."
...,...,...,...
416804,"[i, feel, like, tell, these, horni, devil, to,...",2,"[0.261473, -0.43588337, -0.47987348, -0.606215..."
416805,"[i, began, to, realiz, that, when, i, wa, feel...",3,"[0.14168677, -0.18397643, -0.47203425, -0.1903..."
416806,"[i, feel, veri, curiou, be, whi, previou, earl...",5,"[-0.079134695, 0.19806477, -0.9758066, -0.3512..."
416807,"[i, feel, that, becuas, of, the, tyran, natur,...",3,"[0.041908115, -0.37603053, -0.21473238, -0.522..."


In [20]:
y=dataset['label']
# vstack in numpy is used to combine the two or more tuples or list of array vertically
# vstack in pandas is used to combine two or more dataframes vertically or horizontally
x= np.vstack(dataset['vector'].values)

In [21]:
y.dropna(inplace=True)
y.isnull().sum()

0

In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr=LogisticRegression(max_iter=2000)
lr.fit(x_train,y_train)

y_pred=lr.predict(x_test)
accuracy_lr=accuracy_score(y_test,y_pred)
print(accuracy_lr)


0.6242532568796334


In [26]:
x_train_lstm,x_test_lstm,y_train_lstm,y_test_lstm=train_test_split(dataset['text'],dataset['label'],test_size=0.2,random_state=42)

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense

# Your data
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(x_train_lstm)
x_train_seq = pad_sequences(tokenizer.texts_to_sequences(x_train_lstm), maxlen=200)
x_test_seq  = pad_sequences(tokenizer.texts_to_sequences(x_test_lstm),  maxlen=200)

model = Sequential([
    Embedding(20000, 100, input_length=200),
    Conv1D(128, 5, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    LSTM(64),
    Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(x_train_seq, y_train, validation_data=(x_test_seq, y_test), epochs=5, batch_size=32)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 1362s 130ms/step - accuracy: 0.8542 - loss: 0.3485 - val_accuracy: 0.9253 - val_loss: 0.1321
Epoch 2/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 1308s 125ms/step - accuracy: 0.9291 - loss: 0.1270 - val_accuracy: 0.9277 - val_loss: 0.1268
Epoch 3/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 652s 63ms/step - accuracy: 0.9333 - loss: 0.1143 - val_accuracy: 0.9275 - val_loss: 0.1267
Epoch 4/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 668s 64ms/step - accuracy: 0.9374 - loss: 0.1047 - val_accuracy: 0.9276 - val_loss: 0.1286
Epoch 5/5
10421/10421 ━━━━━━━━━━━━━━━━━━━━ 724s 69ms/step - accuracy: 0.9395 - loss: 0.0981 - val_accuracy: 0.9274 - val_loss: 0.1303


In [34]:
from tensorflow.keras.models import load_model
model.save('emotion_classifier.h5')

In [36]:
import pickle
with open("emotion_tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [49]:
import joblib
joblib.dump(lr_tfidf,'lr_tfidf_model.pkl')

['lr_tfidf_model.pkl']